In [1]:
#import packages
import requests
import json
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import datetime


In [2]:
path = "Park Unit Scraping Information.csv"
parks = pd.read_csv(path, encoding='latin-1')
parks.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Abraham Lincoln Birthplace,Abraham Lincoln Birthplace National Historical...,National Historical Park,KY,SER,ABLI,1.0
1,Acadia,Acadia National Park,National Park,ME,NER,ACAD,1.0
2,Adams,Adams National Historical Park,National Historical Park,MA,NER,ADAM,1.0
3,African Burial Ground,African Burial Ground National Monument,National Monument,NY,NER,AFBG,1.0
4,Agate Fossil Beds,Agate Fossil Beds National Monument,National Monument,NE,MWR,AGFO,1.0


In [3]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/alerts?limit=1000&api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [4]:
id_list = []
url_list = []
title_list = []
parkCode_list = []
description_list = []
category_list = []
relatedRoadEvents_list = []
lastIndexedDate_list = []

for x in df['data']:
    #print(x)
    id_list.append(x['id'])
    url_list.append(x['url']) 
    title_list.append(x['title']) 
    parkCode_list.append(x['parkCode'].upper()) 
    description_list.append(x['description'])
    category_list.append(x['category']) 
    relatedRoadEvents_list.append(x['relatedRoadEvents'])
    lastIndexedDate_list.append(x['lastIndexedDate'])


In [5]:
alerts_df = pd.DataFrame({
    'id': id_list, 
    'url': url_list, 
    'title': title_list,
    'park alpha': parkCode_list,
    'description': description_list,
    'category': category_list,
    'relatedRoadEvents':relatedRoadEvents_list,
    'LastIndexedDate':lastIndexedDate_list
})

In [6]:
alerts_df.head()

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate
0,1CC5DA87-2D0C-4E51-B88C-DB49B7F38957,,"April 15, 2024: Summer Hours Begin",STEA,"Effective Monday, April 15, Steamtown NHS will...",Information,[],2024-03-21 08:39:02.0
1,D343DB17-FB31-4953-AC43-957033416C4B,https://www.nps.gov/grca/planyourvisit/grand-c...,What's Closed and What's Open? (Park Operation...,GRCA,Visitor Center is OPEN from 10 am to 4 pm. Ran...,Park Closure,[],2024-03-21 00:02:31.0
2,75487057-E4F6-4A96-9E5F-71A40F272B2F,https://www.nps.gov/grca/planyourvisit/weather...,"Thursday, March 21, 2024 - Mostly Sunny Today ...",GRCA,"Bright Angel Trail, Plateau Pt. and Tonto Trai...",Park Closure,[],2024-03-21 00:00:41.0
3,82666CCF-577D-4717-9BE5-9E42431B1490,,Prescribed Fire Operations,EVER,Due to recent prescribed fire operations north...,Caution,[],2024-03-20 20:27:48.0
4,B090C622-C867-4B23-9250-F0EBF3B5642D,,Road Closure,SHEN,Skyline Drive is closed northbound from Thornt...,Park Closure,[{'title': 'Thornton Gap Entrance Station to M...,2024-03-20 20:08:33.0


In [7]:
alerts_df.category.value_counts()

Information     352
Park Closure    202
Caution         118
Danger           22
                  2
Name: category, dtype: int64

In [8]:
counter = 0
road_closed_list = []

for x in range(0,len(alerts_df)):
    road_closed = 0
    if "lane closure" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if "road closure" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if "road closed" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if "road is closed" in str(alerts_df['description'][x]).lower():
        road_closed = 1
    if ("road" in str(alerts_df['description'][x]).lower())&("closed" in str(alerts_df['description'][x]).lower()):
        road_closed=1
    road_closed_list.append(road_closed)

    if road_closed == 1:
        print(alerts_df['park alpha'][x])
        print(alerts_df['description'][x])
        counter +=1

alerts_df['road_closed_flag'] = road_closed_list
    


GRTE
Spring Plowing has begun on the Teton Park Road. For safety reasons, visitors may NOT access the Teton Park Road while rotary snow removal equipment and plows are working; the roadway is closed to ALL users during this time.
GRTE
Portions of the Teton Park Road (Taggart Lake TH to Signal Mt.), Moose-Wilson Road (Granite Canyon TH to Death Canyon Road), and other roads are closed to vehicles. Check the Grand Teton Roads page for the latest seasonal and weather related closures.
JOTR
Due to recent rains, the following dirt roads are temporarily closed: Big Horn Pass Road, Queen Vally Road, Keys Ranch Road, and O'Dell Road.
OLYM
Hurricane Ridge Road remains closed Monday through Thursday during the winter season. The road is scheduled to reopen Friday, March 22nd, weather permitting.
PRWI
Scenic Drive will be closed from dawn (6:30am) until around 11:00am. Oak Ridge Campground will be completely inaccessible for vehicles entering or leaving during this time. Parking Lots A, B, C, and

In [9]:
sum(alerts_df['road_closed_flag'])

52

In [10]:
alerts_df2 = alerts_df.merge(parks,left_on='park alpha', right_on="Alpha", how = "left")

In [11]:
alerts_df2['relatedRoadEvents'].value_counts()

TypeError: unhashable type: 'list'

In [12]:
alerts_df2.head()

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate,road_closed_flag,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,1CC5DA87-2D0C-4E51-B88C-DB49B7F38957,,"April 15, 2024: Summer Hours Begin",STEA,"Effective Monday, April 15, Steamtown NHS will...",Information,[],2024-03-21 08:39:02.0,0,Steamtown,Steamtown National Historic Site,National Historic Site,PA,NER,STEA,1.0
1,D343DB17-FB31-4953-AC43-957033416C4B,https://www.nps.gov/grca/planyourvisit/grand-c...,What's Closed and What's Open? (Park Operation...,GRCA,Visitor Center is OPEN from 10 am to 4 pm. Ran...,Park Closure,[],2024-03-21 00:02:31.0,0,Grand Canyon,Grand Canyon National Park,National Park,AZ,IMR,GRCA,1.0
2,75487057-E4F6-4A96-9E5F-71A40F272B2F,https://www.nps.gov/grca/planyourvisit/weather...,"Thursday, March 21, 2024 - Mostly Sunny Today ...",GRCA,"Bright Angel Trail, Plateau Pt. and Tonto Trai...",Park Closure,[],2024-03-21 00:00:41.0,0,Grand Canyon,Grand Canyon National Park,National Park,AZ,IMR,GRCA,1.0
3,82666CCF-577D-4717-9BE5-9E42431B1490,,Prescribed Fire Operations,EVER,Due to recent prescribed fire operations north...,Caution,[],2024-03-20 20:27:48.0,0,Everglades,Everglades National Park,National Park,FL,SER,EVER,1.0
4,B090C622-C867-4B23-9250-F0EBF3B5642D,,Road Closure,SHEN,Skyline Drive is closed northbound from Thornt...,Park Closure,[{'title': 'Thornton Gap Entrance Station to M...,2024-03-20 20:08:33.0,0,Shenandoah,Shenandoah National Park,National Park,VA,NER,SHEN,1.0


In [13]:
alerts_df2.loc[alerts_df2['relatedRoadEvents'].str.contains("nps.gov")==True]

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate,road_closed_flag,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit


In [14]:
alerts_df_closures = alerts_df2.loc[alerts_df2['road_closed_flag']==1]
alerts_df_closures.Reg.value_counts()

IMR    16
PWR    13
SER     7
NER     5
MWR     5
NCR     2
AKR     1
NE      1
Name: Reg, dtype: int64

In [15]:
alerts_df2.to_csv("alerts_api.csv", index=False)